In [2]:
from sqlalchemy import create_engine
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
import datetime
import pandas as pd
import urllib.parse
import mysql


def get_drvier():
#Set options to make browsing easier
  options = webdriver.ChromeOptions()
  options.add_argument("disable-infobars")
  options.add_argument("start-maximized")
  options.add_argument("disable-dev-shm-usage")
  options.add_argument("no-sandbox")
  options.add_experimental_option("excludeSwitches", ["enable-automation"])
  options.add_argument("disable-blink-features=AutomationControlled")

  driver = webdriver.Chrome(options=options)
  driver.get("https://snabb.posist.biz/login")
  return driver


def main():
    
  driver = get_drvier()
  driver.find_element(by="name", value="username").send_keys("entp_admin")
  
  time.sleep(2)
  driver.find_element(
    by="name", value="password").send_keys("Admin@98769" + Keys.RETURN)
  time.sleep(4)
#crm  
  driver.find_element(
    by="xpath", value='''//*[@id="menu-content"]/div/li[9]''').click()
  time.sleep(2)
#reports
  driver.find_element(
    by="xpath", value='''//*[@id="navbar-main"]/ul[1]/li[5]/a''').click()
  time.sleep(2)
#crmreport
  driver.find_element(
    by="xpath", value='''//*[@id="sidebar"]/div/accordion/div/div[5]/div[1]/h4/a/div''').click()
  time.sleep(2)

#crmreport
  driver.find_element(
    by="xpath", value='''//*[@id="sidebar"]/div/accordion/div/div[5]/div[2]/div/ul/li[1]/a''').click()
  time.sleep(2)

#aggregated bill
  driver.find_element(by="xpath", value='''//*[@id="content"]/div/div/div[2]/div/div/div[2]/div/label/input''').click()
  time.sleep(2)

#for cloud kitchen button
  driver.find_element(by="xpath", value='''//*[@id="content"]/div/div/div[2]/div/div/div[1]/div[1]/div/div/a/span[2]''').click()
  time.sleep(2)

  driver.find_element(by="xpath", value='''//*[@id="content"]/div/div/div[2]/div/div/div[1]/div[1]/div/div/div/div/input''').send_keys("Snabb Foods - DLF PHASE 4" + Keys.RETURN)
  time.sleep(2)
  
  yesterday = datetime.date.today() - datetime.timedelta(days=1)
#Format the date range
  date_range = f"{yesterday} - {yesterday}"
  print(date_range)

#Format the date variable
  date = yesterday.strftime("%Y.%m.%d--%Y.%m.%d")
  print(date)

#Clear the existing value in the input box
  input_element = driver.find_element(by="xpath", value='''//*[@id="content"]/div/div/div[2]/div/div/div[1]/div[2]/div/input''')

#Select the existing text in the input box
  input_element.click()
  time.sleep(2)

  driver.find_element(by="xpath", value='''/html/body/div[7]/div[3]/ul/li[2]''').click()   
  time.sleep(3)
    
#Generate Report  
  element = driver.find_element(by="xpath", value='''//*[@id="content"]/div/div/div[2]/div/div/a''')
  driver.execute_script("arguments[0].click();", element)
  time.sleep(5)

  driver.find_element(by="xpath", value='''//*[@id="content"]/div/div/div[3]/div[2]/button''').click()
  time.sleep(1)
#export csv
  driver.find_element(by="xpath", value='''//*[@id="content"]/div/div/div[3]/div[2]/ul/li[1]/a''').click()
  time.sleep(12)
  print("done")
  print(driver.current_url)
print(main())


yesterday = datetime.date.today() - datetime.timedelta(days=1)

#Format the date variable
x = yesterday.strftime("%Y.%m.%d--%Y.%m.%d")
print(x)  # 2023.06.07--2023.06.07

#Specify the file path
file_path = r"C:\Users\AVDHESH\Downloads\Customer_Data_Report({})_Snabb Foods - DLF PHASE 4.csv".format(x)


#Read the CSV file, skipping the first 5 lines and the last 7 lines
data = pd.read_csv(file_path, skiprows=5, skipfooter=8, engine='python')

#Drop the "Unnamed: 26" column
data = data.drop("Unnamed: 26", axis=1)

# Rename the "Bill Number" column to "Bill_Number"
data = data.rename(columns={"Bill Number": "Bill_Number"})
data = data.rename(columns={"Bill Date": "Bill_Date"})
data = data.rename(columns={"Mobile Number": "Mobile_Number"})
data = data.rename(columns={"Phone Number": "Phone_Number"})
data = data.rename(columns={"Postal Code": "Postal_Code"})
data = data.rename(columns={"Item Category": "Item_Category"})
data = data.rename(columns={"Item Name": "Item_Name"})
data = data.rename(columns={"Item Qty": "Item_Qty"})
data = data.rename(columns={"Item Price": "Item_Price"})
data = data.rename(columns={"Item Subtotal": "Item_Subtotal"})
data = data.rename(columns={"Net Sales": "Net_Sales"})
data = data.rename(columns={"Total Tax": "Total_Tax"})
data = data.rename(columns={"Gross Sales": "Gross_Sales"})


#Concatenate "Bill Number" and "Item Name" into a new column
data["Primary_Key"] = data["S.No."].astype(str) + " " + data["Bill_Number"].astype(str) + " " + data["Item_Name"].astype(str)


#Print the updated data to the console
print(data)

#Set database connection details
user = 'admin'
password = 'rr_1234567'
host = 'rr-raw.cybvhc8iyrsv.us-east-1.rds.amazonaws.com'
port = 3306
database = 'snabb_db'

#Encode special characters in password
password_encoded = urllib.parse.quote_plus(password)

#Set chunk size for batch processing (required for large data)
chunksize = 10000

# Load the DataFrame into a CSV file
data.to_csv('snabb.csv', index=False)

#Set up database connection
engine = create_engine(f'mysql+mysqlconnector://{user}:{password_encoded}@{host}:{port}/{database}?connect_timeout=10000000')

#Load the CSV file into a MySQL table in batches
for chunk in pd.read_csv('snabb.csv', chunksize=chunksize, low_memory=False):
    chunk.to_sql(name='snabb_order_table', con=engine, if_exists='append', index=False)

#Close the database connection
engine.dispose()


2023-06-29 - 2023-06-29
2023.06.29--2023.06.29
done
https://snabb.posist.biz/reports/customerDataReport
None
2023.06.29--2023.06.29
     S.No.                     Outlet  Bill Number   Bill Date        Channel  \
0        1  Snabb Foods - DLF PHASE 4         4292  2023-06-29  Capo Delivery   
1        2  Snabb Foods - DLF PHASE 4         4292  2023-06-29  Capo Delivery   
2        3  Snabb Foods - DLF PHASE 4         4292  2023-06-29  Capo Delivery   
3        4  Snabb Foods - DLF PHASE 4         4292  2023-06-29  Capo Delivery   
4        5  Snabb Foods - DLF PHASE 4         4293  2023-06-29  Capo Delivery   
..     ...                        ...          ...         ...            ...   
103    104  Snabb Foods - DLF PHASE 4         4328  2023-06-29  Capo Delivery   
104    105  Snabb Foods - DLF PHASE 4         4328  2023-06-29  Capo Delivery   
105    106  Snabb Foods - DLF PHASE 4         4329  2023-06-29  Wham Delivery   
106    107  Snabb Foods - DLF PHASE 4         4329  2023-0